# Test Kettenregel mit Numpy

In [1]:
import numpy as np
from numpy import ndarray
from typing import *

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#base-functions" data-toc-modified-id="base-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>base functions</a></span></li><li><span><a href="#Verschachtelte-Funktionen" data-toc-modified-id="Verschachtelte-Funktionen-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Verschachtelte Funktionen</a></span></li><li><span><a href="#Kettenregeln" data-toc-modified-id="Kettenregeln-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Kettenregeln</a></span></li><li><span><a href="#Funktionen-mit-mehreren-Eingaben" data-toc-modified-id="Funktionen-mit-mehreren-Eingaben-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Funktionen mit mehreren Eingaben</a></span></li><li><span><a href="#Ableitungen-von-Funktionen-mit-mehreren-Eingaben" data-toc-modified-id="Ableitungen-von-Funktionen-mit-mehreren-Eingaben-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Ableitungen von Funktionen mit mehreren Eingaben</a></span></li></ul></div>

## base functions

In [8]:
def square(x: ndarray) -> ndarray:
    """ Square each element in the input ndarray. """
    return np.power(x, 2)

def deriv(func: Callable[[ndarray], ndarray],
          input_: ndarray,
          diff: float = 0.001) -> ndarray:
    """ Evaluates the derivative of a function "func" at every element in the "input_" array. 
    """
    return (func(input_ + diff) - func(input_ - diff)) / (2 * diff)

def sigmoid(x: ndarray) -> ndarray:
    """Apply the sigmoid function to each element in the input ndarray. """
    return 1 / (1 + np.exp(-x))

In [4]:
nda = np.array([[1,2,3],
                [4,5,6]])

def n(x):
    return x
deriv(n, nda)

array([[1., 1., 1.],
       [1., 1., 1.]])

In [5]:
# A Function takes in an ndarray as an argument and produces an ndarray
Array_Function = Callable[[ndarray], ndarray]

# A Chain is a list of functions
Chain = List[Array_Function]

## Verschachtelte Funktionen

In [6]:
def chain_length_2(chain: Chain,
                   x: ndarray) -> ndarray:
    """ Evaluates two functions in a row, in a "Chain". """
    
    assert len(chain) == 2, \
    "Length of input 'chain' should be 2"

    f1 = chain[0]
    f2 = chain[1]

    return f2(f1(x))

In [7]:
chain_length_2([square, square], nda)

array([[   1,   16,   81],
       [ 256,  625, 1296]])

## Kettenregeln

In [22]:
def chain_deriv_2(chain: Chain,
                  input_range: ndarray) -> ndarray:
    """ Uses the chain rule to compute the derivative of two nested functions:
        (f2(f1(x))' = f2'(f1(x)) * f1'(x)
    """
 
    assert len(chain) == 2, \
    "This function requires 'Chain' objects of length 2"

    assert input_range.ndim == 1, \
    "Function requires a 1 dimensional ndarray as input_range"

    f1 = chain[0]
    f2 = chain[1]

    # df1/dx
    f1_of_x = f1(input_range)
    print(f1_of_x)

    # df1/du
    df1dx = deriv(f1, input_range)
    print(df1dx)

    # df2/du(f1(x))
    df2du = deriv(f2, f1(input_range))
    print(df2du)

    # Multiplying these quantities together at each point
    return df1dx * df2du

In [23]:
na = np.array([1,2,3])
chain_deriv_2([square, square], na)

[1 4 9]
[2. 4. 6.]
[ 2.  8. 18.]


array([  4.,  32., 108.])

In [29]:
def chain_deriv_3(chain: Chain,
                  input_range: ndarray) -> ndarray:
    """ Uses the chain rule to compute the derivative of three nested functions:
        (f3(f2(f1)))' = f3'(f2(f1(x))) * f2'(f1(x)) * f1'(x) 
    """

    assert len(chain) == 3, \
    "This function requires 'Chain' objects to have length 3"

    f1 = chain[0]
    f2 = chain[1]
    f3 = chain[2]
    
    ## forward propagation ##

    # f1(x)
    f1_of_x = f1(input_range)
    print(f1_of_x)

    # f2(f1(x))
    f2_of_x = f2(f1_of_x)
    print(f2_of_x)
    
    
    ## backpropagation ##

    # df3du
    df3du = deriv(f3, f2_of_x)
    print(df3du)

    # df2du
    df2du = deriv(f2, f1_of_x)
    print(df2du)

    # df1dx
    df1dx = deriv(f1, input_range)
    print(df1dx)

    # Multiplying these quantities together at each point
    return df1dx * df2du * df3du

In [30]:
na = np.array([1,1,1])
chain_deriv_3([square, square, square], na)

[1 1 1]
[1 1 1]
[2. 2. 2.]
[2. 2. 2.]
[2. 2. 2.]


array([8., 8., 8.])

## Funktionen mit mehreren Eingaben

In [31]:
def multiple_inputs_add(x: ndarray,
                        y: ndarray,
                        sigma: Array_Function) -> float:
    """ Function with multiple inputs and addition, forward pass. """
    
    assert x.shape == y.shape

    a = x + y
    return sigma(a)

In [39]:
x = np.array([2, 2, 2])
y = np.array([1, 0, 3])
multiple_inputs_add(x, y, square)

array([ 9,  4, 25])

## Ableitungen von Funktionen mit mehreren Eingaben

In [44]:
def multiple_inputs_add_backward(x: ndarray,
                                 y: ndarray,
                                 sigma: Array_Function) -> float:
    """ Computes the derivative of this simple function with respect to both inputs. """
    
    # Compute "forward pass"
    a = x + y

    # Compute derivatives
    dsda = deriv(sigma, a)

    dadx, dady = 1, 1

    return dsda * dadx, dsda * dady

In [45]:
x = np.array([2, 2, 2])
y = np.array([1, 0, 3])
multiple_inputs_add_backward(x, y, square)

(array([ 6.,  4., 10.]), array([ 6.,  4., 10.]))